# Fireveg DB - download data for analysis

Author: [José R. Ferrer-Paris](https://github.com/jrfep) and [Ada Sánchez-Mercado](https://github.com/adasanchez)

Date: January 2025

This Jupyter Notebook includes **R code to download data records** of the data exported from the Fireveg Database. 

Here we will read input from a public data record of the database to save a local copy of the data, in the next workbooks will use these data to answer some questions about the data coverage of the database.

```mermaid
flowchart LR
    Litrev & Form  --> Fireveg --> DR --> Down --> Code --> Q1 & Q2 & Q3
    Fireveg[(Fireveg\nDatabase)]
    Litrev[Field work\ndata stream] 
    Form[Existing sources\n data stream] 
    BioNet(BioNet Atlas\nSpecies list) 
    BioNet -.-> Fireveg
    DR[Exported\nData Record\nVersion 1.1]:::ThisRepo
    Down{R code\nfor data download}:::ThisRepo
    Code{R code\nfor data analysis}
    Q3["Q2. ..."]
    Q2["Q3. ..."]
    Q1["Q1. ..."] 
classDef ThisRepo fill:none,stroke:black,color:black;

```

## Set-up

### Load packages

In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




For data download from cloud storage

In [2]:
require(osfr)
library(jsonlite)
library(httr)

Loading required package: osfr

Automatically registered OSF personal access token

Warning message:
“package ‘jsonlite’ was built under R version 4.3.3”


### Paths for inputs and outputs

Locate the root directory of the repo

In [3]:
here::i_am("Notebooks/00-Data-download.ipynb")

here() starts at /Users/z3529065/proyectos/fireveg/fireveg-analysis



Relative path to local data files within project repository

In [4]:
data_dir <- here::here("data")
if (!dir.exists(data_dir))
    dir.create(data_dir)

### Download data

<div class="alert alert-info">
     <img src='../img/open-data-2.png' width=25 alt="open data icon"/>
Data for this Notebook is available from the following repositories:

> Ferrer-Paris, J. R., Keith, D., & Sánchez-Mercado, A. (2024, August 15). Export data records from Fire Ecology Traits for Plants database. Retrieved from [osf.io/h96q2](https://osf.io/h96q2/)

and 
> Ferrer-Paris, J. R.; Keith, D. (2024). Fire Ecology Traits for Plants: Database exports. figshare. Dataset. Retrieved from <https://doi.org/10.6084/m9.figshare.24125088.v2>
</div>

Here we will download data programmatically from the figshare url and from OSF cloud storage to our local data folder. 

#### Download data from Figshare

We use the figshare url and declare our destination file

In [5]:
url <- "https://figshare.com/ndownloader/articles/24125088/versions/2"
destfile <- here::here(data_dir, "figshare-data.zip")

Ada's suggestion is to use the `GET` function from package `httr`:

In [6]:
if (!file.exists(destfile)) {
    GET(url, write_disk(destfile, overwrite = FALSE), config = config(ssl_verifypeer = FALSE))
}

And then to use `unzip` to create a new folder:

In [7]:
unzip(destfile, exdir = here::here(data_dir, "figshare_data"))

#### Download data from OSF

Here we will download data programmatically from OSF cloud storage to our local data folder. First we will check the metadata for the target file. We use the `osf_ls_files` function from package `osfr` to explore the metadata of the files associated to the OSF component.

In [8]:
osf_project <- osf_retrieve_node("https://osf.io/h96q2")
file_list <- osf_ls_files(osf_project)

This will give us a list of file names and ids:

In [9]:
select(file_list, name, id)

name,id
<chr>,<chr>
fire-history.rds,6452ba9d13904f00b7fc85d2
Quadrat-sample-data.rds,6452bab38ea16b0093b69427
site-visits.rds,6452bac07177850087b0f73c
Summary-traits-family.rds,6452bacfb30b4900b4b9ddc4
Summary-traits-species.rds,6452bae3717785008bb0f4b1
field-sites.gpkg,648a583bbee36d028d0e6261
Summary-traits-sources.rds,64966f6fa2a2f4075a436743
Trait-info.rds,649a64e8a2a2f40aa7436407
References-traits-sources.rds,66c8198039554f1e062d2f46


For a single file we can do:

In [10]:
selected_file <- osf_ls_files(osf_project, pattern="Summary-traits-sources.rds")

And we can explore that file's  metadata:

In [11]:
str(selected_file$meta,3)

List of 1
 $ :List of 3
  ..$ attributes   :List of 16
  .. ..$ guid                    : NULL
  .. ..$ checkout                : NULL
  .. ..$ name                    : chr "Summary-traits-sources.rds"
  .. ..$ kind                    : chr "file"
  .. ..$ path                    : chr "/64966f6fa2a2f4075a436743"
  .. ..$ size                    : int 1195532
  .. ..$ provider                : chr "osfstorage"
  .. ..$ materialized_path       : chr "/Summary-traits-sources.rds"
  .. ..$ last_touched            : NULL
  .. ..$ date_modified           : POSIXct[1:1], format: "2024-08-23 09:57:39"
  .. ..$ date_created            : POSIXct[1:1], format: "2023-06-24 04:22:07"
  .. ..$ extra                   :List of 2
  .. ..$ tags                    : list()
  .. ..$ current_user_can_comment: logi TRUE
  .. ..$ current_version         : int 4
  .. ..$ show_as_unviewed        : logi TRUE
  ..$ links        :List of 8
  .. ..$ info    : chr "https://api.osf.io/v2/files/64966f6fa2a2f4075a4

We can download one file:

In [12]:
RDSfile <- here::here(data_dir,'Summary-traits-sources.rds')

osf_download(selected_file,
             data_dir,
             conflicts = "skip")

name                       id                      
1 Summary-traits-sources.rds 64966f6fa2a2f4075a436743
  local_path                                                                        
1 /Users/z3529065/proyectos/fireveg/fireveg-analysis/data/Summary-traits-sources.rds
  meta                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

Or we can select a subset of files to download

In [13]:
files_to_download <- c(
    "Trait-info.rds",
    "Summary-traits-sources.rds", 
    "References-traits-sources.rds",
    "Quadrat-sample-data.rds",
    "Summary-traits-species-orders.rds"
)

In [14]:
selected_files <- filter(file_list, name %in% files_to_download)

To download the latest version we apply the `osf_download` function with option `conflicts="overwrite"`. 
If we already have the latest version we can choose option `conflicts="skip"`.

In [15]:
downloaded_files <- osf_download(selected_files,
             data_dir,
             conflicts = "overwrite")


#### What about older versions?

We can request more complete version information with a direct call to the API using the `read_json` function. For example the versions for one of the downloaded file are available here:

In [16]:
file_versions <- read_json(downloaded_files$meta[[5]]$relationships$versions$links$related$href)

And we could use these urls to download specific versions:

In [17]:
results <- lapply(file_versions$data, function(x) {
    sprintf("version id %s from %s available at %s\n",
            x$id,
            x$attributes$date_created,
            x$links$download)
})
for (res in results) 
    cat(res)

version id 2 from 2024-08-26T00:24:29.005736 available at https://osf.io/download/66c829ef6725569184c5ca7a/?revision=2
version id 1 from 2024-08-23T06:19:27.104873 available at https://osf.io/download/66c829ef6725569184c5ca7a/?revision=1


### Trait descriptions

```mermaid
flowchart LR
     Fireveg --> DR --> Code --> Q1 & Q2 & Q3
    Q2 --> Q4 & Q5
    Fireveg[(Fireveg\nDatabase)]
   
    DR[Exported\nData Record\nVersion 1.1]
    Code{R code\nfor analysis}:::ThisRepo
    Q1[Q1. Trait descriptions]:::ThisRepo
    Q2["Q2. Trait coverage\n(NSW plant species)"]
    Q3["Q3. ..."]    
    Q4["Q4. ..."]   
    Q5["Q5. ..."]
classDef ThisRepo fill:none,stroke:black,color:black;

```

The data frame `trait_info` includes descriptions of all traits, here we show the priority traits that are already uploaded in the current version of the database.

In [18]:
tbl_trait_info <- trait_info %>% 
  filter(!is.na(priority)) %>%
  rowwise() %>% 
  mutate(Code=code, Trait=name, 
         Description = description,
            `Classification` = paste( 
              life_stage,
              life_history_process, 
              sep="/")) %>%
  ungroup() %>% 
    arrange(desc(life_history_process),Code) %>% 
  select(Code, Trait, Classification, Description) %>%
  knitr::kable()
    
display_markdown(paste(as.character(tbl_trait_info), collapse="\n"))

ERROR: Error: object 'trait_info' not found


### Trait coverage

```mermaid
flowchart LR
     Fireveg --> DR --> Code --> Q1 & Q2 & Q3
    Q2 --> Q4 & Q5 & Q6
    Fireveg[(Fireveg\nDatabase)]
   
    DR[Exported\nData Record\nVersion 1.1]
    Code{R code\nfor analysis}:::ThisRepo
    Q1[Q1. ...]
    Q2["Q2. Trait coverage\n(NSW plant species)"]:::ThisRepo
    Q3["Q3. ..."] 
    Q4["Q4. Range of observed values for each trait"]:::ThisRepo  
    Q5["Q5. ..."]:::ThisRepo  
    Q6["Q6. ..."]:::ThisRepo   
classDef ThisRepo fill:none,stroke:black,color:black;

```

In [ ]:
table(trait_data$`trait code`)

In [ ]:
waffle_plot_trait <- function(x, trait_code, legend_rows = 1) {
    
trait_summary <- x |> 
    dplyr::filter(`trait code` %in% trait_code) |> 
    group_by(`norm value`) |>
    summarise(n = n_distinct(`scientific name`)) 

fig_caption <- sprintf("***%s*** (n = %s)",
                       trait_code, sum(trait_summary$n))

trait_summary |>
ggplot(aes(values = n, fill = `norm value`)) +
  geom_waffle(
    n_rows =10,        # Number of squares in each row
    color = "white",   # Border color
    flip = F, na.rm = TRUE, 
    make_proportional = T,
    show.legend = T) +
    coord_equal() +
    theme_void() +
    theme(legend.position="right", 
          legend.text=element_text(size=14),
          plot.title=element_markdown(size=16)) +
    labs(title=fig_caption) +
    guides(fill=guide_legend(title="", ncol = legend_rows))
}


In [ ]:
trait_code <- "repr4"
trait_summary <- dplyr::filter(trait_data, `trait code` %in% trait_code) 
trait_summary <- trait_summary |> 
     mutate(`norm value`=str_replace_all(`norm value`,"\\[|\\]","")) |>
    separate(`norm value`,
                    into=c("best", "lower", "upper"), sep = ",") |>
    mutate(value = as.numeric(best)) |>
    filter(!is.na(value))
sprintf("***%s*** (n = %s)",
                       trait_code, n_distinct(trait_summary$`scientific name`))
    

In [ ]:
trait_summary

In [ ]:
box_plot_trait <- function(x, trait_code) {
    trait_summary <- dplyr::filter(x, `trait code` %in% trait_code) 
    fig_caption <- sprintf("***%s*** (n = %s)",
                       trait_code, n_distinct(trait_summary$`scientific name`))
    
    trait_summary <- trait_summary |> 
     mutate(`norm value`=str_replace_all(`norm value`,"\\[|\\]","")) |>
    separate(`norm value`,
                    into=c("best", "lower", "upper"), sep = ",") |>
    mutate(best = as.numeric(best),
          lower = as.numeric(lower),
          upper = as.numeric(upper)) |>
    pivot_longer(cols = c("best","lower","upper"),
                 names_to = "bound", values_to = "value")

    
    ggplot(filter(trait_summary, !is.na(value) & bound %in% "best")) +
    geom_histogram(aes(x=value)) +
    #scale_y_log10() +
    theme_minimal() +
    theme(plot.title=element_markdown(size=16)) +
    labs(title=fig_caption) 
    }

In [ ]:
plot_rect2 <- waffle_plot_trait(trait_data, "rect2")
plot_repr2 <- waffle_plot_trait(trait_data, "repr2")
plot_surv4 <- trait_data |> 
    mutate(`norm value`= case_when(`norm value` %in% "Long rhizome or root sucker" ~ "Long rhizome",
                                  TRUE ~ `norm value`)) |> 
    waffle_plot_trait( "surv4")
plot_germ1 <- waffle_plot_trait(trait_data, "germ1", legend_rows = 1)
plot_germ8 <- waffle_plot_trait(trait_data, "germ8")
plot_surv1 <- waffle_plot_trait(trait_data, "surv1", legend_rows = 1)
plot_disp1 <- waffle_plot_trait(trait_data, "disp1", legend_rows = 1)

plot_repr3 <- box_plot_trait(trait_data, "repr3")
plot_repr3a <- box_plot_trait(trait_data, "repr3a")
plot_repr4 <- box_plot_trait(trait_data, "repr4")
plot_surv5 <- box_plot_trait(trait_data, "surv5")

In [ ]:
options(repr.plot.width=16, repr.plot.height=16) # Make plot larger

plot_grid(plot_surv1, plot_surv4, plot_surv5,
          plot_germ1, plot_germ8,
          plot_repr2, plot_repr3, plot_repr3a, plot_repr4,
          plot_rect2,
          plot_disp1, 
           align = "vh",
          labels = NA,
         ncol = 3)


In [ ]:
last_fire <- sites_record |>
  mutate(last_fire = case_when(
      `Time since last fire (days)` %in% c(NA,"0 days","ERROR: mismatching dates") ~ 'unknown',
      !grepl("year", `Time since last fire (days)`) ~ 'recent',
      grepl("^(2 years|3 years|1 year)", `Time since last fire (days)`) ~ 'recent',
      TRUE ~ "older"
  )) |>
  select(`Survey`,`Site label`, `Visit date`, `Time since last fire (days)`, "last_fire")  |>
  rename(visit_id = `Site label`, visit_date = `Visit date`)

In [ ]:
last_fire |> slice_sample(n=5)

In [ ]:
sites_record |> slice_sample(n=5)

In [ ]:
# 3. Calculate traits by species -----
# Number of individuals with a given trait. All the spp in the datset
full_spp_trait <- field_records |>
    filter(!is.na(species_code)) |>
  mutate(spp_type = case_when(
    resprout_organ %in% c("None") ~ "Seeder", 
    TRUE ~ "Resprouter")
  ) |>
 # left_join(species_list, by = c("species" = "Scientific name (as entered)")) |> # Add family information
  left_join(distinct(species_list), by = c("species_code" = "CAPS code")) |> # Add family information
  left_join(last_fire, by = c("visit_id","visit_date")) |>  # Add fire information
  filter(last_fire == "recent") |> # Calculate the metrics only for these sites with time since last fire <= 3 years
  group_by(Family, species, Survey, visit_id, visit_date, spp_type, resprout_organ, seedbank) |>
  summarise(n1 = sum(resprouts_live, na.rm = TRUE),         # N total live resprouts (N1)
            n2 = sum(resprouts_reproductive, na.rm = TRUE), # N reproductive live resprouts (N2)
            n5 = sum(recruits_live, na.rm = TRUE),          # N total live recruits (N5)
            n6 = sum(recruits_reproductive, na.rm = TRUE),  # N reproductive live recruits (N6)
            n7 = sum(resprouts_died, na.rm = TRUE),         # N Dead resprouts (N7). This variable is all 0
            n8 = sum(recruits_died, na.rm = TRUE),          # N dead recruits (N8)
            n9 = sum(resprouts_kill, na.rm = TRUE),      # N fire killed resprouts (N9)
            .groups = "drop"
) |>
  mutate(prop_fire_mortality = n9 /(n1 + n7 + n9),
         prop_sprout_surv = n7/ (n1 + n7),
         seed_adult = (n5 + n8) / (n1 + n7),
         pro_recruit_surv = n5 / (n5 + n8),
         prop_reprod_recruit = n6 / max(n5)
  ) |>
  as_tibble() 


In [ ]:
full_spp_trait |> slice_sample(n=5)

In [ ]:
full_spp_trait |> select(species) |> n_distinct()
full_spp_trait |> select(species,spp_type) |> n_distinct()


In [ ]:
top_species

In [ ]:

# 5. Select the top 5 families -----
# Which are the families with more spp?
top_families <- species_list |>
    filter(!is.na(Family)) |>
  group_by(Family) |>
  summarise(
    n_spp = n_distinct(`Scientific name (as entered)`)
  ) |>
  arrange(desc(n_spp)) |>
  slice_head( n = 5 ) |>
  pull(Family)




In [ ]:
#title = "<b>Distribution of species count by resprout organ<b>",
#       subtitle = "Variation in resprouting strategies among the top 5 families"
plot_organ_type <- full_spp_trait|> 
  filter(Family %in% top_families) |>
  filter(!is.na(resprout_organ)) |>
  group_by(Family, resprout_organ) |>
  summarise(n_species = n_distinct(species), .groups = "drop") |>
  arrange(n_species) |> 
  mutate(resprout_organ = fct_reorder(resprout_organ, desc(n_species))) |>
  ggplot(aes(x = resprout_organ, y = n_species)) +
  geom_bar(stat="identity", fill = "black") +
  facet_grid(~ Family) +
  coord_flip() +
  ylim(0, 60) +
  labs(y = "Number of species",
       x = "") +
  theme_classic() +
  theme(plot.title=element_markdown(), # Enable markdown for title and subtitle
        plot.subtitle=element_markdown())


In [ ]:

       #title = "<b>Distribution of species count by seedbank type<b>",
       #subtitle = "Variation in seedbank strategies among the top 5 families") +
plot_seedbank <- full_spp_trait|> 
  filter(Family %in% top_families) |>
  filter(!is.na(seedbank)) |>
  group_by(Family, seedbank) |>
  summarise(n_species = n_distinct(species), .groups = "drop") |>
  arrange(n_species) |> 
  mutate(seedbank = fct_reorder(seedbank, desc(n_species))) |>
  ggplot(aes(x = seedbank, y = n_species)) +
  geom_bar(stat="identity", fill = "black") +
  facet_grid(~ Family) +
  ylim(0, 60) +
  coord_flip() +
  labs(y = "Number of species",
       x = "") +
  theme_classic() +
  theme(plot.title = element_markdown(), # Enable markdown for title and subtitle
        plot.subtitle = element_markdown())


In [ ]:
options(repr.plot.width=12, repr.plot.height=12) # Make plot larger
plot_grid(plot_organ_type, plot_seedbank, 
           align = "vh",
          labels = "auto",
          rel_heights = c(3, 2),
         ncol = 1)

In [ ]:
library(ggridges)  

In [ ]:

# 4. Select the top 20 spp ----
# Species with more localities and plots
top_species <- full_spp_trait |>
    filter(Survey %in% "Mallee Woodlands") |>
  group_by(species) |>
  summarise(
    n_localities = n_distinct(visit_id),
    n_visits = n_distinct(visit_id,visit_date)
  ) |>
arrange(desc(n_localities)) |>
slice_head(n=20) |> pull(species)


In [ ]:
Mallee_dataset <- full_spp_trait |>
 #filter(species %in% top_species, Survey %in% "Mallee Woodlands") |> # Only the top 20 spp.
 #   filter(species %in% top_species) |> # Only the top 20 spp.
  mutate(species = reorder(species, spp_type, FUN = function(x) ifelse(x[1] == "Seeder", 1, 2)))

In [ ]:
Mallee_dataset |> 
filter(!is.na(prop_fire_mortality), prop_fire_mortality>0) |> 
       select(prop_fire_mortality, species, spp_type)

In [ ]:
?geom_histogram

In [ ]:
ggplot(Mallee_dataset, 
         aes(x = prop_fire_mortality)) +
geom_histogram(aes(y = after_stat(count / sum(count))), bins = 8) +
facet_grid(.~Survey)

In [ ]:
plot_mortality <- 
  ggplot(Mallee_dataset, 
         aes(x = prop_fire_mortality, y = species, fill = spp_type, color = spp_type)) +
  geom_density() +
  theme_ridges() +
  labs(title = "Fire mortality",
       x = "Proportion",
       y = "",
       fill = "Species Type") +
  scale_fill_manual(values = c("Resprouter" = "#9EBCDA", "Seeder" = "#000000")) +
  scale_color_manual(values = c("Resprouter" = "#9EBCDA", "Seeder" = "#000000")) +
  scale_x_continuous(limits = c(0,1), breaks = seq(0, 1, by = 0.5)) +
  theme(legend.position = "none",
        axis.text.y = element_text(face = "italic") )

In [ ]:
plot_recruit_surv <- 
    ggplot(Mallee_dataset,
        aes(x = pro_recruit_surv, y = species, fill = spp_type, color = spp_type)) +
  geom_density_ridges(stat = "binline",
                      bins = 5, draw_baseline = TRUE) +
  theme_ridges() +
  labs(title = "Recruit survival",
       x = "Proportion",
       y = "",
       fill = "Species Type") +
  scale_fill_manual(values = c("#9EBCDA", "#000000")) +
  scale_color_manual(values = c("#9EBCDA", "#000000")) +
  scale_x_continuous(limits = c(0,1), breaks = seq(0, 1, by = 0.5)) +
  theme(legend.position = "none",
        axis.text.y = element_blank())

plot_reprod_recruit <- 
    ggplot(Mallee_dataset,
       aes(x = prop_reprod_recruit, y = species, fill = spp_type, color = spp_type)) +
  geom_density_ridges(scale = 0.4) +
  theme_ridges() +
  labs(title = "Reproductive recruits",
       x = "Proportion",
       y = "",
       fill = "Species Type") +
  scale_fill_manual(values = c("#9EBCDA", "#000000")) +
  scale_color_manual(values = c("#9EBCDA", "#000000")) +
  scale_x_continuous(limits = c(0,1), breaks = seq(0, 1, by = 0.5)) +
  theme(legend.position = "none",
        axis.text.y = element_blank())



In [ ]:
plot_grid(plot_mortality, plot_recruit_surv, plot_reprod_recruit,
          labels = "auto",
          rel_widths = c(2, 1, 1),
         ncol = 3)

In [ ]:

library(patchwork)
plot_mortality + plot_recruit_surv + plot_reprod_recruit 
#+
#  plot_layout(ncol = 3) +
#  plot_annotation(title = 'Distribution of traits in the top 20 spp in the Mallee data',
#                  subtitle = 'Resprouter species in light blue and seeders in black. Only sites with time since the last fire sites ≤3 years were included in traits calculation'
#  ) #&
  #theme(plot.title = element_text(size = 16),
  #      plot.subtitle = element_text(size = 12) )

In [ ]:
full_spp_trait |> count(spp_type)
full_spp_trait |> count(seedbank)
full_spp_trait |> count(resprout_organ)
full_spp_trait |> count(Family)
full_spp_trait |> count(last_fire)

In [ ]:
filter(field_records, visit_date<"2019-01-01") |>
group_by(species) |>
summarise(nsites = n_distinct(visit_id),
          nvisits = n_distinct(visit_id,visit_date),
         nplots = n_distinct(visit_id,visit_date,sample_nr)) |>
arrange(desc(nvisits)) |>
slice_head(n=15)

In [ ]:
filter(field_records, visit_date > "2019-01-01") |>
group_by(species) |>
summarise(nsites = n_distinct(visit_id),
          nvisits = n_distinct(visit_id,visit_date),
         nplots = n_distinct(visit_id,visit_date,sample_nr)) |>
arrange(desc(nvisits)) |>
slice_head(n=15)

In [ ]:
spp_selection <- "Sclerolaena diacantha"
spp_selection <- "Microlaena stipoides"


datos_spp1 <- field_records |> 
    filter(species %in% spp_selection) |>
      group_by(visit_id, visit_date) |>
      summarise(n1 = sum(resprouts_live, na.rm = TRUE),         # N total live resprouts (N1)
                n2 = sum(resprouts_reproductive, na.rm = TRUE), # N reproductive live resprouts (N2)
                n5 = sum(recruits_live, na.rm = TRUE),          # N total live recruits (N5)
                n6 = sum(recruits_reproductive, na.rm = TRUE),  # N reproductive live recruits (N6)
                n7 = sum(resprouts_died, na.rm = TRUE),         # N Dead resprouts (N7)
                n8 = sum(recruits_died, na.rm = TRUE),          # N dead recruits (N8)
                n9 = sum(resprouts_kill, na.rm = TRUE),         # N fire killed resprouts (N9)
                .groups = "drop") |>
  mutate(n_fire_mortality = n1 + n7 + n9,
         prop_fire_mortality = n9 /(n1 + n7 + n9),
         n_sprout_surv = n1 + n7,
         prop_sprout_surv = n7/ (n1 + n7),
         seed_adult = (n5 + n8) / (n1 + n7),
         n_recruit_surv = n5 + n8,
         pro_recruit_surv = n5 / (n5 + n8),
         n_reprod_resprod = n1,
         n_reprod_recruit = n6, # the formula said N5 but this is the number of lives recruits
         prop_reprod_recruit = n6 / max(n5),
         surv_dens = n2 / 625, # plot area
         recruit_dens = n5 / 625 # plot area
         )

In [ ]:
options(repr.plot.width=5, repr.plot.height=5) 
ggplot(datos_spp1) +
geom_histogram(aes(x=prop_fire_mortality))

In [ ]:
options(repr.plot.width=5, repr.plot.height=5) 
ggplot(datos_spp1) +
geom_histogram(aes(x=pro_recruit_surv))

In [ ]:
options(repr.plot.width=5, repr.plot.height=5) 
ggplot(datos_spp1) +
geom_histogram(aes(x=prop_reprod_recruit))

In [ ]:
surv6_data

We will now look at how many species from the NSW BioNet Atlas are represented in the fireveg database. First let's take a look at the NSW flora, and then we will query how many species have information about fire ecology traits from existing sources or field work. We will combine this information in the final subsection.

#### Plant species in NSW according to BioNet Atlas

The data frame `spp_traits_table` is based on the BioNet Altas list of species. 

This list includes around 8170 distinct taxa (based on current taxonomic status) at the species level which are considered native and alive in NSW. It also includes  around 1250 infra-species level taxa considered to be alive in NSW.

In [ ]:
spp_traits_table |>
    group_by(`species level`= taxonrank %in% "Species",establishment) |>
    summarise(records=n(), `original names` = n_distinct(spp), `current names`=n_distinct(current_species), .groups = "drop") 

Here we focus on the current names, that means that information collected under two or more different, older taxonomic names for a single current taxonomic name is interpreted here as being equivalent to redundant information records from a single taxon. It is important to keep this in mind to avoid double counting or inconsistencies in the following queries. 

#### Fire ecology traits from existing sources

First we check the number of taxa with at least one record from existing sources:

In [ ]:
spp_traits_table |>
    filter(`Existing sources`>0) |>
    group_by(`species level`= taxonrank %in% "Species",establishment) |>
    summarise(records=n(), `original names` = n_distinct(spp), `current names`=n_distinct(current_species), .groups = "drop") 

Focusing on taxa at the species level, we want to get an overview of records from the two main sources imported in this data stream. Here we filter the `traits_table` data frame which has information about main sources and the primary sources references by each record.

In [ ]:
summary_per_source <- traits_table |> 
    filter(taxonrank %in% "Species") |>
    group_by(main_source) |>
    summarise(
        records=n_distinct(rid),
        traits = n_distinct(traitcode),
        species=n_distinct(current_species), 
        sources=n_distinct(primary_source))

In [ ]:
summary_per_source

There are a few problems with some sources. The Austraits data includes the NSWFFRD data, so we should exclude this to avoid the most obvious duplicate entries. Other duplicates might still be present, but a manual curation of the source list is needed to flag this. 

Another problem is that some items in the source list do not have reference information in the database (no entries in the `references` data frame. These sources are mentioned in the primary source column, but bibliographic details are incomplete or missing from our database:

In [ ]:
traits_table |> 
    filter(!primary_source %in% references$ref_code) |>
    distinct(primary_source) |> arrange() |> pull()

If we exclude those there is a drop in the number of records and species.

In [ ]:
traits_table |>
    filter(taxonrank %in% "Species") |>
    filter(!primary_source %in% 'Kenny Orscheg Tasker Gill Bradstock 2014', # this is the same as NSWFFRDv2.1
           primary_source %in% references$ref_code) |> # exclude transcription errors
    group_by(main_source) |>
    summarise(
        total=n(), 
        records=n_distinct(rid), 
        species=n_distinct(current_species), 
        sources=n_distinct(primary_source))

#### Field work data

Next, we look at the taxa represented in the field work data. How many species have at least one record in the field sample?

In [ ]:
spp_traits_table |>
    filter(`Field samples`) |>
    group_by(`species level`= taxonrank %in% "Species",establishment) |>
    summarise(records=n(), `original names` = n_distinct(spp), `current names`=n_distinct(current_species), .groups = "drop") 

In order to get more detailed break down of numbers, we use the `quadrat_samples` data frame:

In [ ]:
quadrat_samples |> 
    filter(!is.na(species_code),
           taxonrank %in% "Species") |>
    group_by(survey_name) |>
    summarise(
        locations = n_distinct(visit_id),
        visits  = n_distinct(visit_id, visit_date),
        samples  = n_distinct(visit_id, visit_date, sample_nr),
        records = n(), 
        species = n_distinct(species),
        current_species = n_distinct(current_species), 
        codes = n_distinct(species_code))

The Mallee Woodlands survey is a longer time series of observation in a single region in Western New South Wales, while all the other surveys represent samples completed after the 2019-2020 fires in Eastern New South Wales. We will calculate summary statistics for these two groups:

In [ ]:
summary_per_survey <- quadrat_samples |> 
    filter(!is.na(species_code),
           taxonrank %in% "Species") |>
    group_by(survey_group=survey_name %in% "Mallee Woodlands") |>
    summarise(
        locations = n_distinct(visit_id),
        visits  = n_distinct(visit_id, visit_date),
              samples  = n_distinct(visit_id, visit_date, sample_nr),
              records = n(), 
              species = n_distinct(species),
              current_species = n_distinct(current_species), 
              codes = n_distinct(species_code)) |> 
    arrange(survey_group)

In [ ]:
summary_per_survey

#### Summary from all sources

If we focus only on the taxa at the species level which are native and alive in NSW, we can look at the overlap between both field work data and existing sources:

In [ ]:
spp_traits_table |> 
    filter(
           taxonrank %in% "Species",
           establishment %in% "Alive in NSW, Native") |>
    group_by(`Field samples`, `Existing sources`) |>
    summarise(total = n_distinct(scientific_name), 
              total_current = n_distinct(current_species), 
              .groups = "drop")

And the same for all species (native and introduced):

In [ ]:
spp_traits_table |> 
    filter(
           taxonrank %in% "Species") |>
    group_by(`Field samples`, `Existing sources`) |>
    summarise(total = n_distinct(scientific_name), 
              total_current = n_distinct(current_species), 
              .groups = "drop")

As mentioned above, there might be some overlaps in these numbers due to a species being represented by multiple records with different older/outdated names. Best way to get a total number of current species included in the database is this:

In [ ]:
total_records <- spp_traits_table |> 
    filter(
           taxonrank %in% "Species") |>
    filter(`Field samples` | `Existing sources`) |>
    summarise(total_current=n_distinct(current_species))

In [ ]:
total_records

Is this the same as the sum of the values in the above table?

In [ ]:
5484+100+793

Now we can prepare a summary of all data inputs:

In [ ]:
new_surveys <- quadrat_samples |> filter(!survey_name %in% c("Mallee Woodlands"))
old_surveys <- quadrat_samples |> filter(survey_name %in% c("Mallee Woodlands"))
total_spp <- pull(total_records,total_current)
NSWFFRD_records <- filter(summary_per_source, main_source %in% 'NSWFFRDv2.1')
Austraits_records <- filter(summary_per_source, main_source %in% 'austraits-6.0.0')
Postfire_samples <- filter(summary_per_survey, !survey_group)
Mallee_samples <- filter(summary_per_survey, survey_group)

In [ ]:
tbl = sprintf("
| Type	| Unit of observation	| Spatial information	| Number of records	| Number of taxa (including non-native)	| Data source |
|---|---|---|---|---|---|
| **Primary Observations** |
| Post-fire field surveys | Individual | %s sites | %s | %s  | East coast post-fire surveys 2020-2022 |
| Time series field observations | Individual | %s sites | %s | %s | Mallee vegetation dynamics 2007-2018 [@Keith_Tozer_2012] |
| **Compilations** |
| Fire response 	| Species	| Not applicable	| %s	| %s	 | %s sources compiled in NSW plant fire response database [@Kenny2014] | 
|Species traits	| Individuals / Populations / Species	| Variable |	%s	| %s	 | %s sources compiled in AusTraits plant database [@Falster2021] |
| Total | |  |  | %s  |
", 
              Postfire_samples$locations, Postfire_samples$records, Postfire_samples$current_species,
            Mallee_samples$locations, Mallee_samples$records, Mallee_samples$current_species,
              NSWFFRD_records$records, NSWFFRD_records$species, NSWFFRD_records$sources,
              Austraits_records$records, Austraits_records$species, Austraits_records$sources,
               total_spp
             )

display_markdown(tbl)

Now, we will create a visualisation of represented plant orders based on an example from:
http://wilkox.org/treemapify/

We select from the table all taxa with rank of species, and we group by current scientific name, then we summarise data from field samples and existing sources. We then extract the genus name and categorise species according to the source of the fire ecology traits. We group small orders.


In [ ]:
oos <- table(spp_traits_table$rank_order)
gt_table <- spp_traits_table |>
    filter(taxonrank %in% "Species") |>
    group_by(rank_order, current_species) |>
    summarise(
        `Field samples`=any(`Field samples`),
        `Existing sources`=any(`Existing sources`),
        .groups = "drop") |>
    mutate(
        genus = str_split_i(current_species," ",1),
        fire_ecology_traits_from=case_when(
          `Field samples` & `Existing sources` ~ "both",
          `Field samples` ~ "field",
          `Existing sources` ~ "literature",
          TRUE ~ "none"),    
        rank_order = case_when(
            rank_order %in% names(oos)[oos<10] ~ "small orders",
            is.na(rank_order) ~ "unknown",
            TRUE ~ rank_order
        )
      )
clrs <- c(none="aliceblue", field="yellow", literature="orange", both="maroon")

We check which order are missing data on fire ecology traits:

In [ ]:
dim(gt_table)
# check that any(duplicated(gt_table$current_species)) is FALSE
gt_table |>
    group_by(rank_order,fire_ecology_traits_from) |>
    summarise(total=n(), .groups='drop') |>
    pivot_wider(id_cols=rank_order, 
            names_from = fire_ecology_traits_from, 
            values_from = total) |>
    filter(is.na(both),is.na(field),is.na(literature)) |>
    arrange(none)

[Cornales](https://en.wikipedia.org/wiki/Cornales) is an order of flowering plants, and is only represented by seven species in NSW. 

All other orders are groups of mosses or liverworts, we can ignore these.

- Feather moss:
    - [Hypnales](https://en.wikipedia.org/wiki/Hypnales)
- Liverworts:
    - Marchantiales https://en.wikipedia.org/wiki/Marchantiales
    - Jungermanniales https://en.wikipedia.org/wiki/Jungermanniales
    - Porellales https://en.wikipedia.org/wiki/Porellales
- Mosses:
    - Orthotrichales https://en.wikipedia.org/wiki/Orthotrichaceae
    - Dicranales https://en.wikipedia.org/wiki/Dicranales
    - Pottiales https://en.wikipedia.org/wiki/Pottiales
    - Bryales https://en.wikipedia.org/wiki/Bryales
    - Grimmiales https://en.wikipedia.org/wiki/Grimmiales

In [ ]:
exclude_mosses_liverworts <- c("Hypnales",
                             "Porellales", "Jungermanniales","Marchantiales",
                             "Orthotrichales", "Dicranales", "Pottiales",
                             "Bryales","Grimmiales"
                             )

This is the final plot

In [ ]:
options(repr.plot.width=16,repr.plot.height=16) # Make plot larger


gt_table |>
    filter(!rank_order %in% exclude_mosses_liverworts) |>
ggplot(aes(area=1, fill = fire_ecology_traits_from, label = genus,
                subgroup = rank_order)) +
  geom_treemap() +
  geom_treemap_subgroup_border() +
  scale_fill_manual(values=clrs)

For the publication version of this plot, we do some additional tinkering of the groups so that we can include labels for the largest orders (but drop labels for very small groups):

In [ ]:
gt_table_summary <- gt_table |> 
    filter(!rank_order %in% exclude_mosses_liverworts) |>
    group_by(rank_order) |> 
    summarise(genera = n_distinct(current_species), .groups = "drop") |>
    arrange(desc(genera))
print(gt_table_summary,n=25)

gt_table_ss <- gt_table |>
    filter(!rank_order %in% exclude_mosses_liverworts) |>
    mutate(rank_order = case_when(
        rank_order %in% c("unknown","unplaced") ~ "unplaced",
        TRUE ~ rank_order
        ))
#|>
#    mutate(rank_order = case_when(
#          rank_order %in% c("Asparagales","Asterales","Poales","Fabales","Myrtales","Proteales","Gentianales","Ericales","Lamiales","Sapindales","small orders") ~ rank_order ,
#          rank_order %in% c("unknown","unplaced") ~ "unplaced",
#          nchar(rank_order) < 10 ~ str_replace(rank_order, "ales$", "."),
#          nchar(rank_order) < 20 ~ str_replace(rank_order, "phyllales$", "ph."),
#          rank_order %in% c("Escalloniales") ~ "Esc.",
#          rank_order %in% c("Cucurbitales") ~ "Cuc.",
#          TRUE ~ abbreviate(rank_order)
#        )
#    )

In [ ]:
clrs <- c(none="whitesmoke", literature="#90B2F2", field="#E6AF00", both="#CC79A7")
    
ggplot(gt_table_ss, aes(area=1, fill = fire_ecology_traits_from, label = genus,
                subgroup = rank_order)) +
  geom_treemap() +
  geom_treemap_subgroup_text(
    place = "topleft", 
    grow = F, 
    reflow = T,
    alpha = 0.85, 
    colour = "black", 
    fontface = "italic", 
    min.size = 0) +
  geom_treemap_subgroup_border() +
  scale_fill_manual(values=clrs) +
  labs(fill='Available data') +
  theme(legend.position = "top")

## That is it for now!

✅ Job done! 😎👌🔥

Now the data is in a local folder and we can run the next notebooks to do some analysis.

Or, you can:
- go [back home](../Instructions-and-workflow.ipynb),
- continue navigating the repo on [GitHub](https://github.com/ces-unsw-edu-au/fireveg-analysis)
- continue exploring the repo on [OSF](https://osf.io/h96q2/).
- visit the database at <http://fireecologyplants.net>

### R session information

In [19]:
date()

[1] "Sat Jan 25 18:05:03 2025"

In [20]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: aarch64-apple-darwin20 (64-bit)
Running under: macOS 15.2

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRblas.0.dylib 
LAPACK: /Library/Frameworks/R.framework/Versions/4.3-arm64/Resources/lib/libRlapack.dylib;  LAPACK version 3.11.0

locale:
[1] en_AU.UTF-8/en_AU.UTF-8/en_AU.UTF-8/C/en_AU.UTF-8/en_AU.UTF-8

time zone: Australia/Sydney
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] httr_1.4.7     jsonlite_1.8.9 osfr_0.2.9     dplyr_1.1.4   

loaded via a namespace (and not attached):
 [1] compiler_4.3.1    crayon_1.5.3      tidyselect_1.2.1  Rcpp_1.0.14      
 [5] IRdisplay_1.1     urltools_1.7.3    uuid_1.1-1        fastmap_1.2.0    
 [9] IRkernel_1.3.2    triebeard_0.4.1   here_1.0.1        R6_2.5.1         
[13] generics_0.1.3    curl_5.2.1        knitr_1.49        tibble_3.2.1     
[17

<div class="alert alert-success">
    <b>Ada's joke of the day</b>:

    > What did the unzipped file say to the compressed file? "Wow, you've really let yourself go!"
</div>

